In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from explorer import Explorer

Additionally, Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/faiss/loader.py:28: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numpy.__version__) >= "1.19":
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/setuptools/_distutils/version.py:337: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to 

In [3]:
e = Explorer("llama3.1", "webshop_llama")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Products loaded.

Keys cleaned.

Attributes loaded.

100%|██████████| 1000/1000 [00:00<00:00, 8423.67it/s]


Loaded 6910 goals.


/home/xingkun/webshop/web_agent_site/envs/web_agent_text_env.py:212: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = self._parse_html(html).findAll(text=True)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/gym/envs/registration.py:619: UserWarning: WARN: Env check failed with the following message: You must specify an observation space (cf gym.spaces) cf https://github.com/openai/gym/blob/master/gym/spaces/
You can set `disable_env_checker=True` to disable this check.
  logger.warn(


In [4]:
e.adaptor.initialize_env()

In [5]:
state = e.adaptor.get_state()
action_status = e.adaptor.get_available_actions()
print(state)
print(action_status)

{'url': 'http://127.0.0.1:3000/<session_id>', 'html': 'WebShop Search'}
{'has_search_bar': True, 'clickables': ['search']}


In [6]:
e.adaptor.get_instruction()

"Find me slim fit, machine wash women's jumpsuits, rompers & overalls with short sleeve, high waist, polyester spandex for daily wear with color: green stripe, and size: large, and price lower than 30.00 dollars"

In [7]:
e.get_next_action(state, action_status)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"search[slim fit, machine wash women's jumpsuits, rompers & overalls with short sleeve, high waist, polyester spandex for daily wear with color: green stripe, and size: large, and price: under 30.00]"

In [ ]:
instruction = e.adaptor.get_instruction()
state = e.adaptor.get_state()
action_status = e.adaptor.get_available_actions()
print(instruction)
print(state)
print(action_status)
p = e.adaptor.get_action_prompt(instruction, state, action_status)
print(p)

In [ ]:
LLAMA3_WEBSHOP_SYSTEM_PROMPT = "You are an intelligent exploration agent that navigates through environments to accomplish tasks. Your goal is to analyze the current state, understand the task instruction, and determine the next action to take. Respond with only the action you want to execute, without any additional explanation or formatting."
instruction = "i am looking for x-large, red color women faux fur lined winter warm jacket coat, and price lower than 70.00 dollars"
state = {'url': 'http://127.0.0.1:3000/<session_id>', 'html': 'WebShop Search'}
available_actions = "[search]"
action_status = {'has_search_bar': True, 'clickables': ['search']}

p = e.adaptor.get_action_prompt(instruction, state, action_status)
print(p)

In [ ]:
e.explorer_model.get_next_action(p)

In [ ]:
LLAMA3_WEBSHOP_SYSTEM_PROMPT = "You are an intelligent exploration agent that navigates through environments to accomplish tasks. Your goal is to analyze the current state, understand the task instruction, and determine the next action to take. Respond with only the action you want to execute, without any additional explanation or formatting."
instruction = "i am looking for x-large, red color women faux fur lined winter warm jacket coat, and price lower than 70.00 dollars"
state = {'url': 'http://127.0.0.1:3000/<session_id>', 'html': 'WebShop Search'}
available_actions = "[search]"
action_status = {'has_search_bar': True, 'clickables': ['search']}

prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{LLAMA3_WEBSHOP_SYSTEM_PROMPT} 
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Task Instruction: {instruction}

Current URL: {state['url']}

Current Webpage Display Text: {state['html']}

Available Actions: {available_actions}

Based on the current state and task instruction, what is the next action you should take?

If you want to search, response with "search[<search_query>]"
<|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>
"""

outputs = pipe(prompt, max_new_tokens=64)
full_text = outputs[0]["generated_text"]
print(full_text)

In [ ]:
e.explore()